# Recommender System

# Workshop 1 : วิเคราะห์ความคล้ายของประโยค

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
a = 'beach sea beach'
b = 'Beach sea sea' # ข้อสังเกต ใช้ B ตัวใหญ่

In [3]:
text = [a, b]
cv = CountVectorizer()
count_matrix = cv.fit_transform(text)

print(cv.get_feature_names()) # รายการคำ (ข้อมูล List)
print(count_matrix.toarray()) # เมทริกซ์แสดงจำนวนคำ

['beach', 'sea']
[[2 1]
 [1 2]]


In [4]:
# เพื่อให้ดูง่ายขึ้น นำข้อมูล Matrix มาเก็บใน DataFrame
term_matrix = count_matrix.todense()
cols = cv.get_feature_names()
df = pd.DataFrame(term_matrix, columns=cols, index=['a','b'])
df

,beach,sea
a,2,1
b,1,2


In [5]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(count_matrix)
cosine_sim

array([[1. , 0.8],
       [0.8, 1. ]])

In [6]:
a = 'beach sea beach'
b = 'Beach sea sea Beach beach beach' # ข้อสังเกต ใช้ คำซ้ำมากขึ้น
text = [a, b]
cv = CountVectorizer()
count_matrix = cv.fit_transform(text)
print(cv.get_feature_names())
print(count_matrix.toarray())

['beach', 'sea']
[[2 1]
 [4 2]]


In [7]:
term_matrix = count_matrix.todense()
cols = cv.get_feature_names()
df = pd.DataFrame(term_matrix, columns=cols, index=['a','b'])
df

,beach,sea
a,2,1
b,4,2


In [8]:
cosine_sim = cosine_similarity(count_matrix)
cosine_sim

array([[1., 1.],
       [1., 1.]])

จะเห็นว่าค่าคำตอบคล้ายกันมาก(เข้าใกล้1)

# Workshop 2 : วิเคราะห์ความคล้ายของ 4 ประโยค

In [9]:
a = 'blue skirt dress lady'
b = 'blue shirt lady'
c = 'smartphone android'
d = 'charger smartphone tablet'

text = [a, b, c, d]

cv = CountVectorizer()
count_matrix = cv.fit_transform(text)

term_matrix = count_matrix.todense()
cols = cv.get_feature_names()
df = pd.DataFrame(term_matrix, columns=cols, index=['a', 'b', 'c', 'd'])
df

,android,blue,charger,dress,lady,shirt,skirt,smartphone,tablet
a,0,1,0,1,1,0,1,0,0
b,0,1,0,0,1,1,0,0,0
c,1,0,0,0,0,0,0,1,0
d,0,0,1,0,0,0,0,1,1


In [10]:
cosine_sim = cosine_similarity(count_matrix)
cosine_sim

array([[1.        , 0.57735027, 0.        , 0.        ],
       [0.57735027, 1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.40824829],
       [0.        , 0.        , 0.40824829, 1.        ]])

+++++สิ่งนี้เป็นพื้นฐานสำคัญสำหรับการแนะนำสินค้าและบริการที่คล้ายกัน

# การใช้แนะนำสินค้าที่คล้ายกัน

In [11]:
#ตรวจสอบความคล้ายคลึงทีละคู่
ref_index = 0 # มอง A
similar_score = cosine_sim[ref_index]
similar_score.round(2)

array([1.  , 0.58, 0.  , 0.  ])

a คล้ายกับ b 0.58

In [12]:
ref_index = 2 #มอง C
similar_score = cosine_sim[ref_index]
similar_score.round(2)

array([0.  , 0.  , 1.  , 0.41])

c คล้ายกับ d 0.41

In [13]:
# สมมุติว่าลูกค้ากำลังดูสินค้า c ก็ยึด c เป็นหลัก แล้วทำการเปรียบเทียบกับตัวอื่นๆ
ref_index = 2
similar_score = cosine_sim[ref_index]

# นำค่าลำดับ และ ค่าดัชนีความคล้ายคลึงมารวมกันเป็น list เพื่อใช้สำหรับอ่านค่า (Access)
similar_products = list(enumerate(similar_score)) # จับคู่อันดับโดยตัวแรกเป็น 0 1 2 3 กับ ค่า cosine_sim
similar_products

[(0, 0.0), (1, 0.0), (2, 0.9999999999999998), (3, 0.408248290463863)]

In [14]:
# ทำการจัดเรียงเอาค่า score สูงสุดไว้บนสุด
sorted_recommend = sorted(similar_products,key=lambda x:x[1], reverse=True)[1:]
sorted_recommend

[(3, 0.408248290463863), (0, 0.0), (1, 0.0)]

In [15]:
# วนรอบแนะนำสินค้า
ref_index = 3
print('Top recommend for '+ df.index[ref_index]+
     '\n--------------------------')
for i,element in enumerate(sorted_recommend):
    title = df.index[element[0]] # รัน element ที่รับค่ามาจาก sortedทีละตัว
    print('{:8} {:.3f}'.format(title, sorted_recommend[i][1]))

Top recommend for d
--------------------------
d        0.408
a        0.000
b        0.000


# Workshop 3 : ระบบแนะนำภาพยนตร์

ข้อมูล : import และ โหลดไฟล์ข้อมูลภาพยนตร์ และตรวจสอบเบื้องต้น

In [16]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('data/movie.csv')

In [17]:
df.shape # 4803 rows 24 columns

(4803, 24)

In [18]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [19]:
df.isnull().sum() # หา missing data

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

คอลัมน์ข้อมูล

In [20]:
# หาลำดับ index ของภาพยนตร์ที่ต้องการ โดยค้นชื่อภาพยนตร์
df[df.title.str.contains('transformer',case= False)].title
# case = False ไม่สนใจตัวใหญ่ตัวเล็ก

35     Transformers: Revenge of the Fallen
36         Transformers: Age of Extinction
52          Transformers: Dark of the Moon
111                           Transformers
Name: title, dtype: object

In [21]:
# ตรวจจาก index . overview
df.iloc[111].overview

'Young teenager, Sam Witwicky becomes involved in the ancient struggle between two extraterrestrial factions of transforming robots – the heroic Autobots and the evil Decepticons. Sam holds the clue to unimaginable power and the Decepticons will stop at nothing to retrieve it.'

In [22]:
# ตรวจดูจาก index . keyword
df.iloc[111].keywords

'destroy transformation alien based on toy transformers'

In [23]:
# ดูตัวอย่างหลายๆตัวโดยละเอียด
ids = [35, 36, 52]
for k in ids:
    print('\n\t Title: ', df.iloc[k].title)
    print('overview: ', df.iloc[k].overview) # column overview
    print()
    print('kwd:',df.iloc[k].keywords) # column keywords
    print('----------------------')


	 Title:  Transformers: Revenge of the Fallen
overview:  Sam Witwicky leaves the Autobots behind for a normal life. But when his mind is filled with cryptic symbols, the Decepticons target him and he is dragged back into the Transformers' war.

kwd: egypt sun chaos symbol artifact
----------------------

	 Title:  Transformers: Age of Extinction
overview:  As humanity picks up the pieces, following the conclusion of "Transformers: Dark of the Moon," Autobots and Decepticons have all but vanished from the face of the planet. However, a group of powerful, ingenious businessman and scientists attempt to learn from past Transformer incursions and push the boundaries of technology beyond what they can control - all while an ancient, powerful Transformer menace sets Earth in his cross-hairs.

kwd: sequel alien transformers giant robot robot
----------------------

	 Title:  Transformers: Dark of the Moon
overview:  Sam Witwicky takes his first tenuous steps into adulthood while remaining a 

In [24]:
# ต้องการดูข้อมูลทั้งหมดทุกคอลัมน์
for k in df.columns:
    print(k, ': ',df[k].iloc[111]) # ลำดับ index ภาพยนตร์ที่ต้องการดูข้อมูล
    print('---------------------')

index :  111
---------------------
budget :  150000000
---------------------
genres :  Adventure Science Fiction Action
---------------------
homepage :  http://www.transformersmovie.com/
---------------------
id :  1858
---------------------
keywords :  destroy transformation alien based on toy transformers
---------------------
original_language :  en
---------------------
original_title :  Transformers
---------------------
overview :  Young teenager, Sam Witwicky becomes involved in the ancient struggle between two extraterrestrial factions of transforming robots – the heroic Autobots and the evil Decepticons. Sam holds the clue to unimaginable power and the Decepticons will stop at nothing to retrieve it.
---------------------
popularity :  25.468493
---------------------
production_companies :  [{"name": "Paramount Pictures", "id": 4}, {"name": "DreamWorks SKG", "id": 27}, {"name": "Amblin Entertainment", "id": 56}, {"name": "Di Bonaventura Pictures", "id": 435}, {"name": "Platin

In [25]:
# เก็บชื่อคอลัมน์ไว้ใน features
features = ['keywords', 'cast', 'genres', 'director']
df[features].isnull().sum()

keywords    412
cast         43
genres       28
director     30
dtype: int64

In [26]:
#เนื่องจากมี missing data มากไป ให้เติมNan ด้วย Blank string
for feature in features:
    df[feature] = df[feature].fillna("")

In [27]:
# รวม features
df['combined_features'] = df.genres + '' + df.keywords+''+df.cast+''+df.director

In [28]:
# ตรวจสอบว่าเปลี่ยนแล้ว
df.combined_features.head()
# df.iloc[0].combined_features
# df.combined_features.iloc[0]

0    Action Adventure Fantasy Science Fictioncultur...
1    Adventure Fantasy Actionocean drug abuse exoti...
2    Action Adventure Crimespy based on novel secre...
3    Action Crime Drama Thrillerdc comics crime fig...
4    Action Adventure Science Fictionbased on novel...
Name: combined_features, dtype: object

ประมวลผล

In [29]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

cv = CountVectorizer() 
count_matrix = cv.fit_transform(df.combined_features)

In [30]:
# ตรวจสอบขนาด
count_matrix.shape

(4803, 24518)

In [31]:
#ตรวจดูตารางคำ 10 คำแรก และ 10 คำสุดท้าย
print(cv.get_feature_names()[0:10])
print(cv.get_feature_names()[-10:])

['11', '15th', '17th', '18th', '1910schristopher', '1910sfernanda', '1917warren', '1920sjudy', '1930szac', '1950slogan']
['zukovic', 'zuleikha', 'zuniga', 'zurer', 'zurertimur', 'zvyagintsev', 'zwart', 'zwick', 'zwigoff', 'zylka']


In [33]:
# แสดงตารางความถี่คำ
print(count_matrix.toarray()[:10, 10:30])

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [34]:
cosine_sim = cosine_similarity(count_matrix)
cosine_sim.shape

(4803, 4803)

In [35]:
# ลองแสดงข้อมูล 8 x 8
print(cosine_sim[:8, :8].round(3))

[[1.    0.079 0.091 0.043 0.218 0.087 0.    0.118]
 [0.079 1.    0.043 0.    0.041 0.164 0.    0.037]
 [0.091 0.043 1.    0.093 0.191 0.048 0.    0.215]
 [0.043 0.    0.093 1.    0.044 0.089 0.    0.04 ]
 [0.218 0.041 0.191 0.044 1.    0.136 0.    0.164]
 [0.087 0.164 0.048 0.089 0.136 1.    0.    0.041]
 [0.    0.    0.    0.    0.    0.    1.    0.   ]
 [0.118 0.037 0.215 0.04  0.164 0.041 0.    1.   ]]


แนะนำภาพยนตร์ที่ใกล้เคียง

In [37]:
user_choice = 'Avatar'
ref_index = df[df.title.str.contains(use_choice, case=False)].index[0]
print('user choice ref_index = {}'.format(ref_index))

user choice ref_index = 0


In [38]:
# แนะนำภาพยนตร์ที่คล้ายกัน
similar_movies = list(enumerate(cosine_sim[ref_index]))
similar_movies[:8]

[(0, 0.9999999999999999),
 (1, 0.07856742013183862),
 (2, 0.09128709291752769),
 (3, 0.042562826537937436),
 (4, 0.21759706994462233),
 (5, 0.08703882797784893),
 (6, 0.0),
 (7, 0.11785113019775793)]

In [39]:
# เรียงลำดับให้คะแนน 6 อันดับแรก
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)[1:]
sorted_similar_movies[:6]

[(94, 0.3481553119113957),
 (3158, 0.3333333333333334),
 (56, 0.2886751345948129),
 (1531, 0.28022426915890253),
 (2403, 0.2672612419124244),
 (3208, 0.2611164839335468)]

In [40]:
#ต้องการทราบที่ index 94 คืออะไร
df.title[94]

'Guardians of the Galaxy'

In [42]:
# ให้แสดงชื่อเรื่อง 6(i+2) อันดับแรก
print('Recommend movies for ['+ user_choice +']')
print('---------------------------------------')
for i,element in enumerate(sorted_similar_movies):
    similar_movie_id = element[0]
    similar_movie_title = df.title.iloc[similar_movie_id] # Title
    s_score = element[1]
    print('{:30} ->{:.3f}'.format(similar_movie_title, s_score))
    if i > 4:
        break

Recommend movies for [Avatar]
---------------------------------------
Guardians of the Galaxy        ->0.348
Alien                          ->0.333
Star Trek Beyond               ->0.289
Moonraker                      ->0.280
Aliens                         ->0.267
Star Wars: Clone Wars: Volume 1 ->0.261


# สรุป Code

1. import library ที่ต้องใช้

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

2. การอ่านข้อมูลและการจัดการข้อมูล

In [4]:
# อ่านข้อมูล movie
df = pd.read_csv('data/movie.csv') 
# คอลัมน์ที่ใช้เป็น features
features = ['keywords', 'cast', 'genres', 'director'] 
# จัดการ missing data(NaN) ด้วย Blank string
for feature in features:
    df[feature] = df[feature].fillna('')
# รวมเข้าด้วยกัน
df['combined_features'] = df.genres + '' + df.keywords + '' + df.cast + '' + df.director
#เตรียมเปลี่ยนข้อมูลเป็น matrix
cv = CountVectorizer() # object
count_matrix = cv.fit_transform(df.combined_features) # feed
cosine_sim = cosine_similarity(count_matrix) # พล็อตกราฟและหามุมระหว่าง matrix

3. Test ผ่าน user_choice

In [5]:
user_choice = 'Avatar' # ผู้ชมกำลังดูเรื่อง Avatar อยู่

4. เตรียมการฟังก์ชันจัดเรียงภาพยนตร์ที่คล้ายกัน

In [7]:
ref_index = df[df.title.str.contains(user_choice, case=False)].index[0]
similar_movies = list(enumerate(cosine_sim[ref_index])) # movie index + ค่าความคล้าย
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1],reverse=True)[1:]

5. อ่านค่าแนะนำหนัง n เรื่อง

In [11]:
n = 8 #อยากแนะนำ 8 เรื่อง
n = n-2
print('Recommend movies for ['+ user_choice +']')
print('---------------------------------------')
for i,element in enumerate(sorted_similar_movies): # วนรอบแนะนำ n เรื่อง
    similar_movie_id = element[0] # index ของmovies
    similar_movie_title = df.title.iloc[similar_movie_id] # Title
    s_score = element[1] # ค่าดัชนีความคล้าย
    print('{:30} ->{:.3f}'.format(similar_movie_title, s_score))
    if i > n:
        break
print('---------------------------------------')

Recommend movies for [Avatar]
---------------------------------------
Guardians of the Galaxy        ->0.348
Alien                          ->0.333
Star Trek Beyond               ->0.289
Moonraker                      ->0.280
Aliens                         ->0.267
Star Wars: Clone Wars: Volume 1 ->0.261
Space Dogs                     ->0.250
Planet of the Apes             ->0.245
---------------------------------------
